In [ ]:
#Yelp 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

In [21]:
#load reviews
train = pd.read_csv('yelp_data/yelp_2013_train.csv')
valid = pd.read_csv('yelp_data/yelp_2013_dev.csv')
test = pd.read_csv('yelp_data/yelp_2013_test.csv')

#add review length to data frame
#yelp['text length'] = yelp['text'].str.len()
#random sample for testing purppses
#yelp = yelp.sample(frac=.1)

X_train = train['text']
y_train = train['rating']
X_valid = valid['text']
y_valid = valid['rating']
X_test = test['text']
y_test = test['rating']

In [22]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(268013,)
(33501,)
(33504,)


In [23]:
max_vocab = 10000
bow_transformer = CountVectorizer(max_features=max_vocab).fit(X_train.values.astype('U'))

In [24]:
X_train = bow_transformer.transform(X_train.values.astype('U'))
X_valid = bow_transformer.transform(X_valid.values.astype('U'))
X_test = bow_transformer.transform(X_test.values.astype('U'))

In [60]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)
#X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=101)


In [25]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train, y_train)
y_hat = nb.predict(X_test)

print(accuracy_score(y_test, y_hat))

0.5534563037249284


In [26]:
#SVM
def linSVM(parameters, X, T, fold) : 
    n_folds = 3
    svc = LinearSVC(multi_class='ovr')
    svc_cv = GridSearchCV(svc, parameters, cv=fold, scoring="accuracy", refit=False)
    svc_cv.fit(X, T)
    scores = svc_cv.cv_results_['mean_test_score']
    params = svc_cv.cv_results_['params']
    print('scores:',scores)
   #print('params:',params)
    return svc_cv.best_params_


#Predefined Split for CV
my_test_fold = []
for i in range(X_train.shape[0]):
    my_test_fold.append(-1)
for i in range(X_valid.shape[0]):
    my_test_fold.append(0)

fold = PredefinedSplit(test_fold=my_test_fold)
CV_split_T = np.append(y_train, y_valid)

import scipy.sparse as sp
CV_split_X = sp.vstack((X_train , X_valid))  # NOT np.vstack


print(CV_split_T.shape)
print(CV_split_X.shape)


(301514,)
(301514, 10000)


In [27]:
#Cs = [ 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]
#Cs = [.002, .003, .0045, .0055, .0065]
#Cs = [.00201, .00202, .00203, .00204, .00205, .00206, .00207, .00208, .0209]
#Cs = [.002045, .002047, .002049, .002051, .002053, .002055]
param_grid = {'C': Cs}

#bestParams = linSVM(param_grid, CV_split_X, CV_split_T, fold)

svc = LinearSVC(C = .0205, multi_class='ovr')
svc.fit(X_train, y_train)

y_hat = svc.predict(X_test)
print(accuracy_score(y_test, y_hat))

0.575185052531041


In [29]:


print(confusion_matrix(y_test, y_hat))
print('\n')
print(classification_report(y_test, y_hat))

print(accuracy_score(y_test, y_hat))

[[1939  347  120  179  274]
 [ 688  830  629  528  289]
 [ 208  425 1264 2208  655]
 [  88  114  548 5883 4349]
 [  71   24   87 2402 9355]]


             precision    recall  f1-score   support

          1       0.65      0.68      0.66      2859
          2       0.48      0.28      0.35      2964
          3       0.48      0.27      0.34      4760
          4       0.53      0.54      0.53     10982
          5       0.63      0.78      0.70     11939

avg / total       0.56      0.58      0.56     33504

0.575185052531041
